In [1]:
# 先安裝line bot需要的套件 
# !pip install "line_bot_sdk==1.16.0"
# print('end')

# push訊息簡單範例

In [ ]:
from linebot import LineBotApi
from linebot.models import TextSendMessage
from linebot.models import *

line_bot_api = LineBotApi('Line Bot 的 Channel access token')
user_id = '使用者的 ID' # 不是加好友的ID

line_bot_api.push_message( user_id, TextSendMessage(text='Hello World!') )
print('...End')

# 股票清單計算

In [4]:
# basic
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True) #不要科學記號
pd.set_option('display.float_format',lambda x : '%.4f' % x)
pd.set_option('display.max_rows', 10)

# 獲得股票資訊
import pandas_datareader as pdr
import yfinance as yf

# 繪圖用的套件
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#獲得時間
import datetime as datetime
import time

#talib計算金融技術指標
import talib
from talib import abstract

import requests

today = datetime.datetime.now() 
day_ago = today - datetime.timedelta(days=100)

import pandas as pd
from datetime import date
import pymssql
print('...End')

...End


# 連接資料庫

In [ ]:
conn = pymssql.connect(host='192.168.137.1',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
cursor = conn.cursor()
sql = 'select * from UserList'
df_user = pd.read_sql(sql, conn)     # 1 UserList
sql = 'select * from StrategyList'
df_stg = pd.read_sql(sql, conn)      # 2 StrategyList
sql = 'select * from TodaySignal'
df_signal = pd.read_sql(sql, conn)   # 3 TodaySignal

In [ ]:
stg_name = df_stg['StrategyName'].tolist()
stg_name = [str(x.strip()) for x in stg_name]  #去除空白!!
stg_par = df_stg['Parameter'].tolist() 
stg_par = [str(x.strip()) for x in stg_par]
stg_index = [str(x) for x in df_stg.index.tolist()]
print(stg_name,stg_par)

# 個別使用者的推薦清單

In [ ]:
# 先更新使用者的關注清單
# 將所有使用者關注策略存成list --> all_users_stg
def renew_user():
    global all_users_stg
    all_users_stg = []
    for user_index in range(len(df_user)):          #第0~6位
        one_user = []
        for i in stg_index:                         #[0,1,2]
            if df_user[i].loc[user_index] == 1: 
                one_user.append(i)
                #print( '第',user_index, '個使用者，', stg_name[ int(i) ], '是否用:',df_user[i].loc[user_index])
        all_users_stg.append(one_user)

In [ ]:
renew_user()
all_users_stg

In [ ]:
# 測試 - 某位使用者
df_result = pd.DataFrame()
user_stg = all_users_stg[3]  #某位使用者
for i in [0,1,2]:                                  #符合數量前三
    if len(user_stg) - i > 0 :
        first = df_signal.T.loc[ user_stg ].sum() == len(user_stg) - i   
        df_first = df_signal[first].reset_index()                 #找出符合(user_stg-i)個策略 的標的，並重置index
        df_first.drop(['index'], axis = 1, inplace = True)
        #加上符合的數量
        df_first = pd.concat( [ df_first, pd.DataFrame([len(user_stg) - i] * len(df_first), columns = ['符合數量'])], axis = 1) 
        df_result = pd.concat([ df_result, df_first], axis = 0 )        #合併
        print(len(user_stg) - i, user_stg)

df_result.reset_index(inplace = True)
df_result.drop(['index'], axis = 1, inplace = True)
df_result

In [ ]:
def Daily_push():
    global all_users_stg_
    # 主程式
    for user_index in range(len(df_user)):                   # 迴圈每位使用者
        # 某位使用者
        df_result = pd.DataFrame()
        user_stg = all_users_stg[user_index]
        for i in range(len(df_signal.columns) - 3):                               #符合數量前三
              if len(user_stg) - i > 0 :
                first = df_signal.T.loc[ user_stg ].sum() == len(user_stg) - i   
                df_first = df_signal[first].reset_index()                         #找出符合(user_stg-i)個策略 的標的，並重置index
                df_first.drop(['index'], axis = 1, inplace = True)
                #加上符合的數量
                df_first = pd.concat( [ df_first, pd.DataFrame([len(user_stg) - i] * len(df_first), columns = ['符合數量'])], axis = 1) 
                df_result = pd.concat([ df_result, df_first], axis = 0 )          #合併

        df_result.reset_index(inplace = True)
        df_result.drop(['index'], axis = 1, inplace = True)

        a_str = ''
        for stg_i in range(len(all_users_stg[user_index])):
            a_str = a_str + ' ' + stg_name[int(all_users_stg[user_index][stg_i])]

        # 建構清單 flex的格式 
        stock_list = []

        for i in range(len(df_result)):                        # 迴圈每檔股票
            column_list = []
            if i >= 150:
                break
            for c in ['股票代號','股票名稱','符合數量']:         # 迴圈一檔股票的每個欄位
                column_list.append({
                    "type": "text",
                    "text": str(df_result[c].loc[i]),
                    "contents": []
                  })        
            stock_list.append({
                "type": "box",
                "layout": "horizontal",
                "contents": column_list
              })

        # 一位使用者的 Flex 訊息
        contents = {
          "type": "bubble",
          "direction": "ltr",
          "footer": {
            "type": "box",
            "layout": "vertical",
            "contents": [
              {
                "type": "box",
                "layout": "vertical",
                "backgroundColor": "#F3B8B8FF",
                "contents": [
                  {
                    "type": "text",
                    "text": df_user['暱稱'].loc[user_index]+"的股票關注清單",
                    "color": "#000000FF",
                    "align": "center",
                    "contents": []
                  }
                ]
              },
              {
                "type": "text",
                "text": "關注策略："+a_str,
                "contents": []
              },
              {
                "type": "box",
                "layout": "horizontal",
                "backgroundColor": "#FCE5C0FF",
                "contents": [
                  {
                    "type": "text",
                    "text": "股票代號",
                    "contents": []
                  },
                  {
                    "type": "text",
                    "text": "名稱",
                    "contents": []
                  },
                  {
                    "type": "text",
                    "text": "符合數量",
                    "contents": []
                  }
                ]
              },        
            ] + stock_list
          }
        }

        message = FlexSendMessage(alt_text="hello", contents=contents)
        line_bot_api.push_message( df_user['userID'].loc[user_index], message )   

In [ ]:
# 推播訊息
conn = pymssql.connect(server='104.199.210.142', user='sqlserver', password='sqlserver', database='LineBot', charset="utf8")
cursor = conn.cursor()
sql = 'select * from 使用者資料'
df_user = pd.read_sql(sql, conn)
Daily_push()

# 各式訊息type